In [1]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Ensure you have downloaded the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to preprocess text
def preprocess_text(text):
    # Tokenize text
    words = word_tokenize(text.lower())
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# Function to extract keywords using TF-IDF
def extract_keywords(text, top_n=5):
    vectorizer = TfidfVectorizer(max_features=top_n, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    keywords = vectorizer.get_feature_names_out()
    return keywords

# Function to summarize text
def summarize_text(text, sentence_count=3):
    sentences = sent_tokenize(text)
    word_frequencies = Counter(word_tokenize(text.lower()))
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies:
        word_frequencies[word] /= max_frequency
    sentence_scores = {
        sentence: sum(word_frequencies.get(word, 0) for word in word_tokenize(sentence.lower()))
        for sentence in sentences
    }
    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary = ' '.join(sorted_sentences[:sentence_count])
    return summary

# Function to process JSON and generate summary
def process_json_and_summarize(json_file, sentence_count=3, keyword_count=5):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Combine all content from sections and summary
    text_content = data.get('Summary', '') + ' '
    for section in data.get('Sections', []):
        text_content += section.get('Content', '') + ' '

    # Preprocess text
    cleaned_text = preprocess_text(text_content)

    # Generate summary
    summary = summarize_text(cleaned_text, sentence_count)

    # Extract keywords
    keywords = extract_keywords(cleaned_text, keyword_count)

    result = {
        "Title": data.get('Title', 'Unknown'),
        "URL": data.get('URL', 'Unknown'),
        "Summary": summary,
        "Keywords": keywords
    }

    return result

# Main function
def main():
    # Input JSON file
    json_file = input("Enter the path to the Wikipedia JSON file: ").strip()

    # Generate summary and keywords
    result = process_json_and_summarize(json_file)

    # Output the result
    print("\n=== Summary ===")
    print(result["Summary"])
    print("\n=== Keywords ===")
    print(", ".join(result["Keywords"]))
    print("\n=== Source URL ===")
    print(result["URL"])

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to C:\Users\yashswi
[nltk_data]     shukla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\yashswi
[nltk_data]     shukla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



=== Summary ===
trump commonly refers donald trump born united states trump card games playing card given high rank trump may also refer trump dog pug owned english painter william hogarth trump horse australian racehorse donald trump foundation charity trump organization business conglomerate founded trump shuttle airline callsign trump trump surname including list people fictional characters name trump gamer jeffrey shih born hearthstone trump colorado trump maryland trump islands antarctica trump islands newfoundland labrador canada trump street city london

=== Keywords ===
born, card, donald, islands, trump

=== Source URL ===
https://en.wikipedia.org/wiki/Trump


In [6]:
import json
from transformers import pipeline

# Function to preprocess and summarize JSON content using transformers
def summarize_with_transformer(json_file, max_summary_length=1500):
    # Load the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Combine all content from the summary and sections
    text_content = data.get('Summary', '') + ' '
    for section in data.get('Sections', []):
        text_content += section.get('Content', '') + ' '

    # Initialize the summarization pipeline
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # Generate a summary
    summary = summarizer(text_content, max_length=max_summary_length, min_length=30, do_sample=False)[0]['summary_text']

    result = {
        "Title": data.get('Title', 'Unknown'),
        "URL": data.get('URL', 'Unknown'),
        "Summary": summary
    }

    return result

# Main function
def main():
    # Input JSON file
    json_file = input("Enter the path to the Wikipedia JSON file: ").strip()

    # Summarize using the transformer model
    try:
        result = summarize_with_transformer(json_file)

        # Output the results
        print("\n=== Title ===")
        print(result["Title"])
        print("\n=== Summary ===")
        print(result["Summary"])
        print("\n=== Source URL ===")
        print(result["URL"])

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


An error occurred: index out of range in self


In [7]:
import openai

# Function to summarize large text
def summarize_large_text(text, model="gpt-3.5-turbo", max_chunk_length=1000):
    # Split text into smaller chunks if it's too long
    text_chunks = [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]
    
    summaries = []
    
    for chunk in text_chunks:
        response = openai.Completion.create(
            model=model,
            prompt=f"Summarize the following text:\n\n{chunk}",
            temperature=0.5,
            max_tokens=150
        )
        summaries.append(response.choices[0].text.strip())
    
    # Combine all summaries into a final summary
    final_summary = "\n".join(summaries)
    
    return final_summary

# Sample usage with the large India text
text = """{large India text from the user request}"""
summary = summarize_large_text(text)

print("Summary:", summary)


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
